# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Assignment 7: Decision Trees

---
** Setup **

In [227]:
require 'test/unit/assertions'
require 'daru'
require 'distribution'
require 'json'

include Test::Unit::Assertions

## Loads data files
def read_sparse_data_from_csv prefix
  data = []
  classes = Hash.new {|h,k| h[k] = 0}
  header = File.read(prefix + ".header").chomp.split(",")  
  
  File.open(prefix + ".csv").each_line.with_index do |l, i|
    a = l.chomp.split ","
    next if a.empty?
    row = {"features" => Hash.new}
    
    header.each.with_index do |k, i|
      v = a[i].to_f
      if k == "label"
        row["label"] = v.to_i
      else
        next if v.zero?
        row["features"][k] = v
      end
    end
    classes[row["label"]] += 1
    
    data << row
  end
  return {"classes" => classes, "features" => header[0,header.size - 1], "data" => data}
end

:read_sparse_data_from_csv

In [228]:
#Preview 2 lines from the Iris dataset
iris = read_sparse_data_from_csv "iris"
iris["data"][0,2]

[{"features"=>{"sepal_length"=>7.0, "sepal_width"=>3.2, "petal_length"=>4.7, "petal_width"=>1.4}, "label"=>1}, {"features"=>{"sepal_length"=>5.7, "sepal_width"=>3.0, "petal_length"=>4.2, "petal_width"=>1.2}, "label"=>1}]

In [229]:
#Preview 2 lines from the Spambase dataset
spambase = read_sparse_data_from_csv "spambase"
spambase["data"][0,2]

[{"features"=>{"word_freq_our"=>0.27, "word_freq_mail"=>0.83, "word_freq_you"=>0.27, "word_freq_your"=>0.27, "word_freq_font"=>8.58, "char_freq_["=>0.092, "char_freq_$"=>0.185, "char_freq_#"=>0.232, "capital_run_length_average"=>7.313, "capital_run_length_longest"=>99.0, "capital_run_length_total"=>607.0}, "label"=>1}, {"features"=>{"word_freq_your"=>0.9, "word_freq_george"=>0.9, "word_freq_data"=>0.9, "char_freq_["=>0.14, "capital_run_length_average"=>3.472, "capital_run_length_longest"=>28.0, "capital_run_length_total"=>125.0}, "label"=>0}]

---
## Question 1.1 (10 Points)
Write a function that calculates the class distribution, $p(c)$, for all classes $c$ for a dataset. 

In [230]:
def class_distribution dataset
  # BEGIN YOUR CODE
  class_group = dataset.group_by{|row| row["label"]}
  class_dist = Hash.new
  ## size of rows
  total_size = dataset.size
  class_num = class_group.size
  
  class_group.each_key do |key|
    class_dist[key] = (class_group[key].size.to_f / total_size.to_f).to_f
  end
  class_dist
  #END YOUR CODE
end

:class_distribution

In [231]:
### TESTS ###
# Check that there are three classes
t1_iris_dist = class_distribution iris["data"]
t1_iris_num_classes = 3
assert_equal(t1_iris_num_classes, t1_iris_dist.size)

In [232]:
### TESTS ###
# Check that each class has a probability 1/3
t1_iris_dist = class_distribution iris["data"]
t1_iris_num_classes = 3
t1_iris_num_classes.times do |cls|
  assert_in_delta t1_iris_dist[cls], 0.33333, 1e-4
end

3

In [233]:
### TESTS ###
t1_spambase_dist = class_distribution spambase["data"]
t1_spambase_num_classes = 2
assert_equal(t1_spambase_num_classes, t1_spambase_dist.size)

In [234]:
### TESTS ###
t1_spambase_dist = class_distribution spambase["data"]
assert_in_delta t1_spambase_dist[0], 0.6, 1e-1
assert_in_delta t1_spambase_dist[1], 0.4, 1e-1

## Question 2 (20 Points)
Adapting your code for calculating the entropy from [Assignment 1](../assignment-1/assignment-1.ipynb), use input from a hash table rather than an array. 

In [260]:
def entropy dist
  # BEGIN YOUR CODE
  dist_copy = dist.clone
  sum = 0.0
  dist_size = dist.size
#   puts dist_size
  dist_size.times do |i|
    if dist_copy[i] != nil
      sum += dist_copy[i]
    end
  end

  if  sum == 0
    return 0.0
  end

  
  dist_size.times do |i|
    if dist_copy[i] != nil
      dist_copy[i] /= sum
    end
  end
  
  entropy = 0.0
  dist_size.times do |i|
    if dist_copy[i] != nil
      if dist_copy[i] == 0.0 
        next
      end
      entropy -= (dist_copy[i]) * Math.log(dist_copy[i])
    end
  end

  entropy
  # END YOUR CODE
end

:entropy

In [261]:
### TESTS ###
#Checks the class entropy for Iris dataset
t2_iris_dist = class_distribution iris["data"]
t2_iris_entropy = entropy t2_iris_dist
assert_in_delta(1.0986122886681096, t2_iris_entropy, 1e-4)

In [262]:
### TESTS ###
#Checks the class entropy for Spambase dataset
t2_spambase_dist = class_distribution spambase["data"]
t2_spambase_entropy = entropy t2_spambase_dist
assert_in_delta(0.6705230209876485, t2_spambase_entropy, 1e-4)

In [263]:
### TESTS ###
# Checks that code handles a class with zero members of a class
t2_zero_dist = {0 => 0.0, 1 => 107.0}
t2_zero_entropy = entropy t2_zero_dist
assert_in_delta(0.0, t2_zero_entropy, 1e-4)

## Question 3.1 (5 points)

Implement information gain given an initial entropy $H_0$ and a hash of counts, where the key of each hash is a split criterion and the value is an array of all examples matching the split. Call the ```class_distribution``` and ```entropy``` functions you have defined above.

The formula for information gain is as follows:

## $IG(Q, V) = H_0 - \sum_{v\in V} \frac{|v|}{|V|} H(c\mid v) $ 

In [264]:
def information_gain h0, splits
  # BEGIN YOUR CODE
  information_gain = 0.0
  
  total_size = 0.0
  splits.each do |key, value|
    total_size += value.size
  end

  entropy_sum = 0.0
  splits.each do |key, value|
    class_dist = class_distribution value
    class_size = value.size
    class_entropy = entropy class_dist
    entropy_sum -= (class_size.to_f / total_size.to_f) * class_entropy.to_f
  end
  
  information_gain = h0 + entropy_sum
  information_gain
  
  #END YOUR CODE
end

:information_gain

In [265]:
### TEST: Random Split ###
t3_random_split = iris["data"].group_by {|row| rand > 0.5 ? "l" : "r"}
t3_entropy = entropy(class_distribution(iris["data"]))


1.0986122886681096

## Question 3.2 (5 points)

The test above verifies the information gain when the iris data is split with uniform probability. What should the information gain be here if we were to do this for the **spambase** dataset?

In [ ]:
## Make this function return a constant equal to your answer to the question above.
def t3_random_information_gain_spambase
  # BEGIN YOUR CODE
  #    spambase = read_sparse_data_from_csv "spambase"
#   t3_random_split = iris["data"].group_by {|row| rand > 0.5 ? "l" : "r"}
#   dist = class_distribution spambase["data"]
#   entropy_0 = entropy dist
#   ig = information_gain entropy_0, t3_random_split
#   ig
  
  4.888371475386677e-06
  #END YOUR CODE
end

In [ ]:
### TEST (Hidden) ###
assert_true(t3_random_information_gain_spambase() >= 0)

## Question 3.3 (10 points)

The test below verifies the information gain of the iris data when split by class value. What should the information gain be here if we were to do this for the **spambase** dataset?

In [243]:
### TEST: Perfect split (by class)
t3_perfect_split = iris["data"].group_by {|row| row["label"]}
t3_perfect_information_gain = information_gain t3_entropy, t3_perfect_split

1.0986122886681096

In [244]:
## Make this function return a constant equal to your answer to the question above.
def t3_perfect_information_gain_spambase
  # BEGIN YOUR CODE
  0.6705230209876485
#    spambase = read_sparse_data_from_csv "spambase"
#   t3_perfect_split = spambase["data"].group_by {|row| row["label"]}
#   dist = class_distribution spambase["data"]
#   entropy_0 = entropy dist
#   ig = information_gain entropy_0, t3_perfect_split
#   ig
  #END YOUR CODE
end

:t3_perfect_information_gain_spambase

In [245]:
### TEST (Hidden) ###

assert_true (t3_perfect_information_gain_spambase() > 0) 

## Question 4 (20 points)
Considering the only the figure below, which shows the iris dataset colored by class, what value of petal_length would have the best information gain?

In [ ]:
d = iris["data"]
df = Daru::DataFrame.new({
  x1: d.collect{|r| r["features"]["petal_length"]}, 
  x2: d.collect{|r| r["features"]["sepal_width"]}, 
  label: d.collect{|r| r["label"]}
}).to_category(:label).plot(type: :scatter, x: :x1, y: :x2, categorized: {by: :label, method: :color}) do |plot, diagram|
  plot.x_label iris["features"][2]
  plot.y_label iris["features"][1]
  plot.legend true
end

In [ ]:
## Make this function return a constant equal to your answer to the question above.
def t4_best_split_for_petal_length_guess
  # BEGIN YOUR CODE
  3
  
  # END YOUR CODE
end

In [ ]:
### TEST (Hidden) ###

## Question 5 (20 points)
Implement a function which takes a feature name and a real-valued threshold and splits data into two groups: 
* Strictly less than the threshold
* Greater than or equal to the threholds. 

This function returns a structure that can be used to calculate the information gain. An example is as follows:
```
{"petal_length < X1":[
        {"features":{"sepal_length":4.8,"sepal_width":3.1,"petal_length":1.6,"petal_width":0.2},"label":0},
],"petal_length >= X1":[
        {"features":{"sepal_length":4.8,"sepal_width":3.1,"petal_length":1.6,"petal_width":0.2},"label":0},
]}
```

In [249]:
def split_on_numeric_value x, k, v
  # BEGIN YOUR CODE
  res = Hash.new
  key_1 = k.to_s + "<" + v.to_s
  key_2 = k.to_s + ">=" + v.to_s
  
  res[key_1] = Array.new
  res[key_2] = Array.new
  
  x.each do |row|
    if row["features"][k] != nil
      if row["features"][k] < v
        res[key_1] << row
      else
        res[key_2] << row
      end
    else
      res[key_1] << row
    end
  end
  res
  
  #END YOUR CODE
end

:split_on_numeric_value

In [250]:
### TEST ###

# Checks for the number of examples when split on petal_length = 1.7
t5_iris_splits = split_on_numeric_value iris["data"], "petal_length", 1.7
t5_split_sizes = t5_iris_splits.values.collect {|v| v.size}.sort
t5_num_splits = 2
assert_equal t5_num_splits, t5_split_sizes.size
assert_equal 44, t5_split_sizes[0]
assert_equal 106, t5_split_sizes[1]

In [251]:
### TEST ###

# Checks the information gain for this split
t5_iris_entropy = entropy(class_distribution(iris["data"]))
t5_iris_information_gain = information_gain t5_iris_entropy, t5_iris_splits
assert_in_delta 0.48280104455013506, t5_iris_information_gain, 5e-2

In [252]:
### TEST ###

# Checks the information gain for a split on the spambase dataset
t5_spambase_splits = split_on_numeric_value spambase["data"], "char_freq_$", 0.056
t5_spambase_entropy = entropy(class_distribution(spambase["data"]))
t5_spambase_information_gain = information_gain t5_spambase_entropy, t5_spambase_splits
assert_in_delta 0.17012249631509135, t5_spambase_information_gain, 5e-2

In [253]:
### TEST ###
sorted_values = iris["data"].collect {|r| r["features"]["petal_length"]}.uniq.sort
min_sl = sorted_values.first
max_sl = sorted_values.last

x = []
y = []
h0 = entropy(class_distribution(iris["data"]))
sorted_values.each do |t|
  x << t
  y << information_gain(h0, split_on_numeric_value(iris["data"], "petal_length", t))
end

assert_equal 43, sorted_values.size
Daru::DataFrame.new({x: x, y: y}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Petal Length Threshold"
  plot.y_label "Information Gain"
  plot.xrange [min_sl,max_sl]
end

#<Nyaplot::Plot:0x00000000041f25f8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000000004191c58 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"a2058b0d-8870-4c15-afbe-32a9b3742eeb"}, @xrange=[1.0, 6.9], @yrange=[0.0, 1.0986122886681096]>], :options=>{:x_label=>"Petal Length Threshold", :y_label=>"Information Gain", :xrange=>[1.0, 6.9], :zoom=>true, :width=>700, :yrange=>[0.0, 1.0986122886681096]}}>

## Question 6 (10 points)
Finding the best split value by calling ```split_on_numeric_value``` on each threshold is _very_ expensive. There is a better solution. The function should return an array consisting of the best threshold and the information gain of that threshold.


In order to find the best split, we need to find $t^\star$ defined as follows:

$ t^\star = \arg \max_{t} IG(x,V(t)) $

Given that a threshold is a single real value, we can simply calculate the values $IG(x,V)$ for every value of $t$ _in order_, which leads allows us to save some work by calculating information incrementally. Notice that the split sets have a specialized form, which we can write as $V(t) = \left\{v_L(t), v_R(t)\right\}$ with $v_L(t) = \left\{x \mid x < t \right\}$ and $v_R(t) = \left\{x \mid x \ge t \right\}$. 

As we sweep $t$ from minimum to maximum values, we note the following relations:

$v_R(t + \Delta_t) = v_R(t) - v_\Delta$

$v_L(t + \Delta_t) = v_L(t) \cup v_\Delta$

where $v_\Delta = \left\{x \mid t < x \le t + \Delta_t \right\}$

What we add to $v_L$ is exactly what was removed from $v_R$, which means that we can easily recalculate the information gain by incrementing a class-specific counter for $v_L$ and decrementing a class-specific counter for $v_R$ as we sweep through the range of the threshold.

The function below returns an array with the split point and the information gain:
```ruby
return [t_max, ig_max]
```


In [254]:
def find_split_point_numeric x, h0, fname
  # BEGIN YOUR CODE
  t_max = 0.0
  ig_max = 0.0
  
  split_l = Hash.new(0)
  split_r = Hash.new(0)
  
  sorted_x = x.sort_by{|row| row["features"].has_key?(fname) ? row["features"][fname] : 0}
  
  sorted_x.each do |row|
    split_r[row["label"]] += 1
  end
  size = sorted_x.size
  ig_max = 0
  sorted_x.each_with_index do |row, index|
    split_l[row["label"]] += 1
    split_r[row["label"]] -= 1
    if(index + 1 < size and row["features"][fname] == sorted_x[index + 1]["features"][fname])
      next
    end
   
    p1 = (index + 1.0)/ size
    p2 = (size - index - 1.0)/ size
    ig = h0 - p1 * entropy(split_l) - p2 * entropy(split_r)
    if ig > ig_max
      ig_max = ig
      t_max = sorted_x[index + 1]["features"][fname]
    end
  end
  
  return [t_max, ig_max]
  #END YOUR CODE
end

:find_split_point_numeric

In [255]:
### TEST ###
t6_iris_entropy = entropy(class_distribution(iris["data"]))
t6_split_sepal_width = find_split_point_numeric iris["data"], t6_iris_entropy, "sepal_width"
assert_in_delta 3.4, t6_split_sepal_width[0], 1e-2

In [256]:
### TEST ###
assert_in_delta 0.18570201019349364, t6_split_sepal_width[1], 1e-2

In [257]:
### TEST ###
t6_spambase_entropy = entropy(class_distribution(spambase["data"]))
t6_split_dollar = find_split_point_numeric spambase["data"], t6_spambase_entropy, "char_freq_$"
assert_in_delta 0.056, t6_split_dollar[0], 1e-2

In [258]:
### TEST ###
assert_in_delta 0.17012249631509135, t6_split_dollar[1], 1e-2

In [259]:
### TEST: Produces same result as split_on_numeric_value ###
t6_iris_expected_ig = information_gain(t6_iris_entropy, split_on_numeric_value(iris["data"], "sepal_width", 3.4))
assert_in_delta t6_iris_expected_ig, t6_split_sepal_width[1], 1e-2
t6_spambase_expected_ig = information_gain(t6_spambase_entropy, split_on_numeric_value(spambase["data"], "char_freq_$", 0.056))
assert_in_delta t6_spambase_expected_ig, t6_split_dollar[1], 1e-2